# Project - Analysis of FBI NICS Firearm Background Checks

## Introduction

In [ ]:
# import packages
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from scipy.stats import pearsonr

# set pandas options
pd.options.mode.chained_assignment = None
pd.set_option('display.max_rows', 1000)
pd.set_option('display.min_rows', 100)
pd.set_option('display.max_columns', 100)
pd.set_option('display.width', 1000)
pd.set_option("expand_frame_repr", True)

# magic function to render plot in notebook
%matplotlib inline

## Data Wrangling

### Loading Data & Preliminary Review

For this project we'll be working with three different datasets
We will load each one and review the dataframe to see if any cleaning is necessary

#### US States Reference Table

In [ ]:
# load dataset into pandas dataframe
dfStates = pd.read_csv('data/us-states.csv')

In [ ]:
# check the df shape (1.1)
dfStates.shape

In [ ]:
# check data types (1.2)
dfStates.dtypes

In [ ]:
# preview df (1.3)
dfStates.head()

#### FBI Firearm Data

In [ ]:
# load dataset into pandas dataframe
dfGunData = pd.read_csv('data/fbi-gun-data.csv')

In [ ]:
# check the df shape (2.1)
dfGunData.shape

In [ ]:
# check data types (2.2)
dfGunData.dtypes

In [ ]:
# preview df
dfGunData.head()

#### US Census Data

In [ ]:
# load dataset into pandas dataframe
dfCensus = pd.read_csv('data/us-census-data.csv')

In [ ]:
# check the df shape (3.1)
dfCensus.shape

In [ ]:
# check data types
dfCensus.dtypes

In [ ]:
# preview df (3.2)
dfCensus.head()

### Cleaning Data

In [ ]:
# function to clean column headers
def CLEAN_COLUMN_HEADERS(df):
    df.columns = df.columns.str.lower()  # change headers to lowercase
    df.columns = df.columns.str.strip()  # remove leading/trailing whitespace
    df.columns = df.columns.str.replace(' ', '_')  # replace spaces with underscores
    print(df.columns, '\n')

# function to perform quick group-by tallies
def TALLY(df, col, export=False):
    tally = df.groupby(col).size().sort_values(ascending=False).reset_index(name='count')

    if export:
        tally.to_csv('tally-output.csv', index=False)
    else:
        print(tally, '\n')

#### US States Reference Table

In [ ]:
# clean column headers (4.1)
CLEAN_COLUMN_HEADERS(dfStates)

#### FBI Firearm Data

In [ ]:
# clean column headers
CLEAN_COLUMN_HEADERS(dfGunData)

# create year column based on existing month column
dfGunData['year'] = dfGunData['month'].str[:4]

# create list of columns in df
colsGunData = dfGunData.columns.tolist()

# move year column in list order, then align df with column list
colsGunData = colsGunData[-1:] + colsGunData[:-1]
dfGunData = dfGunData[colsGunData]

# print columns in df (4.2)
dfGunData.columns

#### US Census Data

In [ ]:
# remove the rows after the actual metrics
dfCensus = dfCensus.loc[:63]

In [ ]:
# rename fact column to 'metric'
dfCensus = dfCensus.rename(columns={'Fact': 'metric'})

# create metric_id
dfCensus.reset_index(level=0, inplace=True)
dfCensus['metric_id'] = dfCensus['index'] + 1
del dfCensus['index']

# create a list of states to use as id_vars when melting dfCensus
stateList = dfStates['name'].unique()

# reshape dataframe into long format (melt)
dfCensus = pd.melt(dfCensus,
                   id_vars=['metric_id', 'metric'],
                   value_vars=stateList,
                   var_name='state',
                   value_name='value')

# clean column headers (4.3)
CLEAN_COLUMN_HEADERS(dfCensus)

In [ ]:
# create list of value flags to remove before converting to numeric
valueFlags = 'D F FN NA S X Z'.split()

# create lists of conditions and results for vectorization method
conditions = [
    dfCensus['value'].str.endswith('%'),
    dfCensus['value'].str.startswith('$') & dfCensus['value'].str.contains(',', na=False),
    dfCensus['value'].str.startswith('$'),
    dfCensus['value'].str.contains(',', na=False),
    dfCensus['value'].isin(valueFlags)
]

resultsValueType = [
    'percent',
    'currency',
    'currency',
    'number',
    'value_flag'
]

resultsValue = [
    dfCensus.value.str[:-1].str.strip(),
    dfCensus.value.str[1:].str.replace(',', '').str.strip(),
    dfCensus.value.str[1:].str.strip(),
    dfCensus.value.str.replace(',', '').str.strip(),
    np.NaN
]

# create value_type using vectorization lists (5.1)
dfCensus['value_type'] = np.select(conditions, resultsValueType, default='number')
TALLY(dfCensus, 'value_type')

# create new_value using vectorization lists (5.2)
dfCensus['new_value'] = np.select(conditions, resultsValue, default=dfCensus.value)
TALLY(dfCensus, 'new_value')

In [ ]:
# convert new_value to numeric
dfCensus['new_value'] = pd.to_numeric(dfCensus['new_value'])
dfCensus.dtypes

# adjust percent values using vectorization
dfCensus['value_percent_fix'] = np.where(
    dfCensus.value_type == 'percent',  # parameter
    dfCensus.new_value / 100,  # true branch
    dfCensus.new_value)  # false branch

# assign cleaned value field to 'value' and drop extra fields
dfCensus['value'] = dfCensus.value_percent_fix
dfCensus = dfCensus.drop(columns=['new_value', 'value_percent_fix'])

### Create Focused Metric Table for Analysis

In [ ]:
# Total Gun Registrations (2016)

# filter fbi gun df to 2016
mask = (dfGunData['year'] == '2016') & (dfGunData['state'].isin(stateList))
dfGunTotals = dfGunData[mask]

# combine totals, grouping by state
dfGunTotals = dfGunTotals[['state', 'totals']]
dfGunTotals = dfGunTotals.groupby(['state'], as_index=False).sum()

# rename totals column
dfGunTotals = dfGunTotals.rename(columns={'totals': 'total_gun_registrations'})

# rearrange and trim dataframe
dfGunTotals = dfGunTotals[['state', 'total_gun_registrations']]

# preview df (6.1)
dfGunTotals.head()

In [ ]:
# function to get individual census metrics by metric_id
def CENSUS_METRICS(metric_id, value):

    # filter census df
    mask = dfCensus['metric_id'] == metric_id
    df = dfCensus[mask]

    # reset index
    df.reset_index(inplace=True)

    # rename value column
    df = df.rename(columns={'value': value})

    # rearrange and trim dataframe
    df = df[['state', value]]

    return df

In [ ]:
# get individual census metrics

# Population Estimates (2016) - 'Population estimates, July 1, 2016,  (V2016)'
dfCensus1 = CENSUS_METRICS(1, 'population_estimate')

# Population by Race - 'Black or African American alone, percent, July 1, 2016,  (V2016)'
dfCensus14 = CENSUS_METRICS(14, 'percent_of_population_black_african_american')

# Population by Race - 'American Indian and Alaska Native alone, percent, July 1, 2016,  (V2016)'
dfCensus15 = CENSUS_METRICS(15, 'percent_of_population_american_indian')

# Population by Race - 'Asian alone, percent, July 1, 2016,  (V2016)'
dfCensus16 = CENSUS_METRICS(16, 'percent_of_population_asian')

# Population by Race - 'Native Hawaiian and Other Pacific Islander alone, percent, July 1, 2016,  (V2016)'
dfCensus17 = CENSUS_METRICS(17, 'percent_of_population_pacific_islander')

# Population by Race - 'Two or More Races, percent, July 1, 2016,  (V2016)'
dfCensus18 = CENSUS_METRICS(18, 'percent_of_population_two_or_more_races')

# Population by Race - 'Hispanic or Latino, percent, July 1, 2016,  (V2016)'
dfCensus19 = CENSUS_METRICS(19, 'percent_of_population_hispanic')

# Population by Race - 'White alone, not Hispanic or Latino, percent, July 1, 2016,  (V2016)'
dfCensus20 = CENSUS_METRICS(20, 'percent_of_population_white')

# Population by Education Level - 'Bachelor's degree or higher, percent of persons age 25 years+, 2011-2015'
dfCensus36 = CENSUS_METRICS(36, 'percent_of_population_bachelors_degree')

# Median Household Income - 'Median household income (in 2015 dollars), 2011-2015'
dfCensus48 = CENSUS_METRICS(48, 'median_household_income')

# Percent of Population in Poverty - 'Persons in poverty, percent'
dfCensus50 = CENSUS_METRICS(50, 'percent_of_population_in_poverty')

# Median Home Value - 'Median value of owner-occupied housing units, 2011-2015'
dfCensus26 = CENSUS_METRICS(26, 'median_home_value')

In [ ]:
# merge metric dataframes

# create list of metric dataframes
metricDataFrames = [
    dfGunTotals,
    dfCensus1,
    dfCensus14,
    dfCensus15,
    dfCensus16,
    dfCensus17,
    dfCensus18,
    dfCensus19,
    dfCensus20,
    dfCensus36,
    dfCensus48,
    dfCensus50,
    dfCensus26
]

# create master metrics dataframe
dfMetrics = dfStates[['name']].drop_duplicates()
dfMetrics.reset_index(inplace=True, drop=True)
dfMetrics = dfMetrics.rename(columns={'name': 'state'})

# merge individual metric dataframes to master metric df
for df in metricDataFrames:
    dfMetrics = dfMetrics.merge(df, on='state')

# preview df (6.2)
dfMetrics.head()

## Exploratory Data Analysis

In [ ]:
# function to create simple scatterplot
def SCATTERPLOT(df, metric1, metric2, title):
    plt.figure(figsize=(8, 8))
    sns.set(font_scale=1)

    scatter = sns.scatterplot(
        data=df, x=metric1, y=metric2
    )

    scatter.set(title=title)

# function to print pearsons correlation
def CORR_VALUE(df, metric1, metric2):
    corr, _ = pearsonr(df[metric1], df[metric2])
    print('Correlation: %.3f' % corr)

# function to create correlation matrix heatmaps
def CORR_MATRIX(df, title):
    plt.figure(figsize=(8, 8))
    sns.set(font_scale=1)

    corr = sns.heatmap(
        df.corr(), vmin=-1, vmax=1, center=0, square=True, annot=True,
        cmap=sns.diverging_palette(20, 220, n=200)
    )
    
    corr.set_xticklabels(
        corr.get_xticklabels(), rotation=45, horizontalalignment='right'
    )

    corr.set(title=title)

### Which states have the most gun registrations per capita?

In [ ]:
# calculate gun registrations per capita
dfMetrics['guns_per_capita'] = dfMetrics['total_gun_registrations'] / dfMetrics['population_estimate']

# create narrow df with state and guns_per_capita
dfRankByState = dfMetrics[['state','guns_per_capita']]
dfRankByState.head()

# sort df in descending order
dfRankByState = dfRankByState.sort_values(
    by=['guns_per_capita'],
    ascending=False
)

# reset index
dfRankByState.reset_index(inplace=True, drop=True)

# create ranking
dfRankByState.reset_index(level=0, inplace=True)
dfRankByState['ranking'] = dfRankByState['index'] + 1
del dfRankByState['index']

# reorder dataframe
dfRankByState = dfRankByState[['ranking', 'state', 'guns_per_capita']]

# print rankings (7.1)
dfRankByState

In [ ]:
# create bar plot to visualize rankings

# adjust plot settings
plt.figure(figsize=(8, 10))
sns.set(font_scale=1)

# create bar plot
bar = sns.barplot(
    data=dfRankByState,
    x='guns_per_capita',
    y='state',
    color='royalblue'
)

# set plot title
bar.set(title='Total Gun Registrations by State (2016)')

In [ ]:
# drop Kentucky
dfNoKentucky = dfMetrics.drop([16])

### Are there any positive correlations between gun registrations and race?

In [ ]:
# create dataframe with metrics pertaining to race

# create list us census metrics relating to race
cols = [
    'guns_per_capita',
    'percent_of_population_black_african_american',
    'percent_of_population_american_indian',
    'percent_of_population_asian',
    'percent_of_population_pacific_islander',
    'percent_of_population_two_or_more_races',
    'percent_of_population_hispanic',
    'percent_of_population_white'
]

# filter dataframe to include only those metrics
dfRaceMetrics = dfNoKentucky[cols]

# preview df (8.1)
dfRaceMetrics.head()

In [ ]:
# look for any correlations with matrix heatmap (8.2)
CORR_MATRIX(dfRaceMetrics, 'Correlation Matrix - US Census Population by Race (2016)')

In [ ]:
# guns_per_capita & percent_of_population_white

# print pearsons correlation value (8.3)
CORR_VALUE(dfRaceMetrics, 'guns_per_capita', 'percent_of_population_white')

# create scatterplot (8.4)
SCATTERPLOT(
    df=dfRaceMetrics, 
    metric1='guns_per_capita', 
    metric2='percent_of_population_white',
    title='Percent of Population White vs. Gun Registrations per Capita'
)

### What is the relationship between higher education and gun registrations?

In [ ]:
# create dataframe with guns per capita and percentage of population with bachelors degree

# create narrow df and preview (9.1)
dfEducationMetric = dfNoKentucky[['guns_per_capita', 'percent_of_population_bachelors_degree']]
dfEducationMetric.head()

In [ ]:
# guns_per_capita & percent_of_population_bachelors_degree

# print pearsons correlation value (9.2)
CORR_VALUE(dfEducationMetric, 'guns_per_capita', 'percent_of_population_bachelors_degree')

# create scatterplot (9.3)
SCATTERPLOT(
    df=dfEducationMetric, 
    metric1='guns_per_capita', 
    metric2='percent_of_population_bachelors_degree',
    title='Percent of Population with Bachelors Degree vs. Gun Registrations per Capita'
)

### 4. What is the relationship between poverty and gun registrations?

In [ ]:
# create dataframe with income, home value, and poverty metrics

# create list us census metrics relating to poverty
cols = [
    'guns_per_capita',
    'median_household_income',
    'percent_of_population_in_poverty',
    'median_home_value'
]

# create narrow df
dfPovertyMetrics = dfNoKentucky[cols]

# preview df (10.1)
dfPovertyMetrics.head()

In [ ]:
# look for any correlations with matrix heatmap (10.2)
CORR_MATRIX(dfPovertyMetrics, 'Correlation Matrix - Poverty, Income, and Home Value (2016)')

In [ ]:
# guns_per_capita & median_home_value

# print pearsons correlation value (10.3)
CORR_VALUE(dfPovertyMetrics, 'guns_per_capita', 'median_home_value')

# create scatterplot (10.4)
SCATTERPLOT(
    df=dfPovertyMetrics, 
    metric1='guns_per_capita', 
    metric2='median_home_value',
    title='Median Home Value vs. Gun Registrations per Capita'
)

In [ ]:
# guns_per_capita & median_household_income

# print pearsons correlation value (10.5)
CORR_VALUE(dfPovertyMetrics, 'guns_per_capita', 'median_household_income')

# create scatterplot (10.6)
SCATTERPLOT(
    df=dfPovertyMetrics, 
    metric1='guns_per_capita', 
    metric2='median_household_income',
    title='Median Household Income vs. Gun Registrations per Capita'
)